# Demonstration of the Granite RAG Context Relevance Intrisic

This notebook shows the usage of the IO processor for the Granite RAG context relevance intrisic, 
also known as the [LoRA Adapter for Context Relevance Classifier]()

This notebook can run its own vLLM server to perform inference, or you can host the 
models on your own server. To use your own server, set the `run_server` variable below
to `False` and set appropriate values for the constants 
`openai_base_url`, `openai_base_model_name` and `openai_lora_model_name`.

In [20]:
from granite_io.io.granite_3_3.input_processors.granite_3_3_input_processor import (
    Granite3Point3Inputs,
)
from granite_io import make_backend
from granite_io.backend.vllm_server import LocalVLLMServer
from granite_io.io.context_relevancy import ContextRelevancyIOProcessor
from granite_io.io.rag_agent_lib import obtain_lora

In [22]:
# Constants go here
base_model_name = "ibm-granite/granite-3.3-8b-instruct"
lora_model_name = "context_relevancy"
run_server = True

In [ ]:
if run_server:
    # Start by firing up a local vLLM server and connecting a backend instance to it.
    # Download and cache the model's LoRA adapter.
    lora_model_path = obtain_lora(lora_model_name)
    print(f"Local path to LoRA adapter: {lora_model_path}")
    server = LocalVLLMServer(
        base_model_name, lora_adapters=[(lora_model_name, lora_model_path)]
    )
    server.wait_for_startup(200)
    lora_backend = server.make_lora_backend(lora_model_name)
    backend = server.make_backend()
else:  # if not run_server
    # Use an existing server.
    # Modify the constants here as needed.
    openai_base_url = "http://localhost:55555/v1"
    openai_api_key = "granite_intrinsics_1234"
    openai_base_model_name = base_model_name
    openai_lora_model_name = lora_model_name
    backend = make_backend(
        "openai",
        {
            "model_name": openai_base_model_name,
            "openai_base_url": openai_base_url,
            "openai_api_key": openai_api_key,
        },
    )
    lora_backend = make_backend(
        "openai",
        {
            "model_name": openai_lora_model_name,
            "openai_base_url": openai_base_url,
            "openai_api_key": openai_api_key,
        },
    )

In [24]:
# Create an example chat completion with a short conversation.
# Base conversation about pets
base_messages = [
    {
        "role": "assistant",
        "content": "I'm here to help you prepare for your job interview!",
    },
    {
        "role": "user",
        "content": "I have a job interview next week for a marketing manager position.",
    },
    {
        "role": "assistant",
        "content": (
            "Congratulations! Marketing manager is an exciting role. "
            "How are you feeling about it?"
        ),
    },
    {
        "role": "user",
        "content": (
            "I'm nervous because I haven't interviewed in years, "
            "and this is a big career move for me."
        ),
    },
    {
        "role": "assistant",
        "content": (
            "It's natural to feel nervous, but preparation will help "
            "boost your confidence."
        ),
    },
    {
        "role": "user",
        "content": (
            "What should I expect them to ask about my experience with "
            "social media campaigns as a marketing manager?"
        ),
    },
]

## Relevant Document Context Relevance Check

In [25]:
# Example 1: RELEVANT document - directly addresses outdoor pets and flea risk
chat_input_relevant = Granite3Point3Inputs.model_validate(
    {
        "messages": base_messages,
        "documents": [
            {
                "text": "Marketing manager interviews often focus on campaign "
                "experience and measurable results. "
                "Expect questions about social media ROI, audience engagement "
                "metrics, and conversion rates. "
                "Prepare specific examples of campaigns you've managed, including "
                "budget, timeline, and outcomes. "
                "Interviewers may ask about your experience with different social "
                "media platforms and their unique audiences. "
                "Be ready to discuss how you measure campaign success and adjust "
                "strategies based on performance data. "
                "Knowledge of current social media trends and emerging platforms "
                "demonstrates industry awareness."
            }
        ],
        "generate_inputs": {"temperature": 0.0},
    }
)

In [ ]:
io_proc = ContextRelevancyIOProcessor(backend)
# Pass our example input through the I/O processor and retrieve the result
chat_result = await io_proc.acreate_chat_completion(chat_input_relevant)
print(chat_result.results[0].next_message.model_dump_json(indent=2))

## Partially Relevant Context Relevance Check

In [27]:
chat_input_partially_relevant = Granite3Point3Inputs.model_validate(
    {
        "messages": base_messages,
        "documents": [
            {
                "text": "Job interviews typically follow a structured format with "
                "behavioral and technical questions. "
                "Preparing specific examples using the STAR method helps answer "
                "behavioral questions effectively. "
                "Research the company's mission, values, and recent news before "
                "your interview. "
                "Dress appropriately for the company culture and arrive 10-15 "
                "minutes early. "
                "Prepare thoughtful questions to ask the interviewer about the role "
                "and company. "
                "Following up with a thank-you email within 24 hours shows "
                "professionalism and interest."
            }
        ],
        "generate_inputs": {"temperature": 0.0},
    }
)

In [ ]:
io_proc = ContextRelevancyIOProcessor(backend)
# Pass our example input through the I/O processor and retrieve the result
chat_result = await io_proc.acreate_chat_completion(chat_input_partially_relevant)
print(chat_result.results[0].next_message.model_dump_json(indent=2))

## Irrelevant Context Check

In [29]:
chat_input_irrelevant = Granite3Point3Inputs.model_validate(
    {
        "messages": base_messages,
        "documents": [
            {
                "text": "Proper knife skills are fundamental to efficient cooking and "
                "food safety in the kitchen. "
                "Different cuts like julienne, brunoise, and chiffonade serve "
                "specific culinary purposes. "
                "Sharp knives are actually safer than dull ones because they require "
                "less pressure to cut. "
                "Learning to properly hold and control a chef's knife takes practice "
                "and patience. "
                "Professional chefs can prep vegetables much faster due to their "
                "refined knife techniques. "
                "Regular knife maintenance including sharpening and proper storage "
                "extends blade life."
            }
        ],
        "generate_inputs": {"temperature": 0.0},
    }
)

In [ ]:
io_proc = ContextRelevancyIOProcessor(backend)
# Pass our example input through the I/O processor and retrieve the result
chat_result = await io_proc.acreate_chat_completion(chat_input_irrelevant)
print(chat_result.results[0].next_message.model_dump_json(indent=2))

## Additional Example on Gardening

In [31]:
gardening_messages = [
    {
        "role": "assistant",
        "content": "Hi! I'd love to help with your gardening questions.",
    },
    {
        "role": "user",
        "content": "I just started a vegetable garden in my backyard this spring.",
    },
    {
        "role": "assistant",
        "content": "That's wonderful! What vegetables are you growing?",
    },
    {
        "role": "user",
        "content": (
            "I planted tomatoes, peppers, and some lettuce. Everything was doing "
            "great until recently."
        ),
    },
    {"role": "assistant", "content": "Oh no, what's been happening with your plants?"},
    {
        "role": "user",
        "content": (
            "The tomato leaves are turning yellow and dropping off. Is this a disease?"
        ),
    },
]

### Gardening Example With Relevant Document

In [32]:
gardening_relevant = Granite3Point3Inputs.model_validate(
    {
        "messages": gardening_messages,
        "documents": [
            {
                "text": "Yellow leaves on tomato plants can indicate several diseases "
                "or conditions. "
                "Early blight causes yellowing leaves that develop brown spots and "
                "eventually drop off. "
                "Fusarium wilt starts with yellowing of lower leaves and progresses "
                "upward. "
                "Overwatering can also cause yellowing as roots become waterlogged "
                "and unable to absorb nutrients. "
                "Nitrogen deficiency typically shows as yellowing starting from the "
                "bottom leaves. "
                "Proper diagnosis requires examining the pattern of yellowing and any "
                "accompanying symptoms."
            }
        ],
        "generate_inputs": {"temperature": 0.0},
    }
)

In [ ]:
io_proc = ContextRelevancyIOProcessor(backend)
# Pass our example input through the I/O processor and retrieve the result
chat_result = await io_proc.acreate_chat_completion(gardening_relevant)
print(chat_result.results[0].next_message.model_dump_json(indent=2))

### Gardening Example with Irrelevant Document

In [34]:
gardening_irrelevant = Granite3Point3Inputs.model_validate(
    {
        "messages": gardening_messages,
        "documents": [
            {
                "text": "Restoring antique furniture requires careful assessment of "
                "the wood type and existing finish. "
                "Stripping old paint or varnish should be done in a well-ventilated "
                "area with proper safety equipment. "
                "Sanding between coats ensures a smooth final finish on wooden "
                "surfaces. "
                "Wood stain penetrates deeper than paint and highlights the natural "
                "grain patterns. "
                "Professional restoration can increase the value of valuable antique "
                "pieces. "
                "Regular maintenance with appropriate wood polish helps preserve "
                "restored furniture."
            }
        ],
        "generate_inputs": {"temperature": 0.0},
    }
)

In [ ]:
io_proc = ContextRelevancyIOProcessor(backend)
# Pass our example input through the I/O processor and retrieve the result
chat_result = await io_proc.acreate_chat_completion(gardening_irrelevant)
print(chat_result.results[0].next_message.model_dump_json(indent=2))

### Gardening Example With Partially Relevant Document

In [36]:
gardening_partial = Granite3Point3Inputs.model_validate(
    {
        "messages": gardening_messages,
        "documents": [
            {
                "text": "Successful vegetable gardening requires attention to soil "
                "quality, watering, and plant spacing. "
                "Different vegetables have varying sunlight and water requirements "
                "throughout the growing season. "
                "Regular inspection of plants helps identify potential problems "
                "before they become serious. "
                "Healthy soil with good drainage supports strong root development "
                "in all garden plants. "
                "Crop rotation prevents soil depletion and reduces disease buildup "
                "in garden beds. "
                "Organic mulch helps retain moisture and suppress weeds around "
                "vegetable plants."
            }
        ],
        "generate_inputs": {"temperature": 0.0},
    }
)

In [ ]:
io_proc = ContextRelevancyIOProcessor(backend)
# Pass our example input through the I/O processor and retrieve the result
chat_result = await io_proc.acreate_chat_completion(gardening_partial)
print(chat_result.results[0].next_message.model_dump_json(indent=2))

In [ ]:
# Free up GPU resources
if "server" in locals():
    server.shutdown()